In [ ]:
import os
import cv2
import random
from PIL import Image
import shutil

In [17]:
# augmentation for flipping FX & FY %15 EACH

def flip_images(image_folder, output_folder_hor, output_folder_ver, percentage=0.15):
    os.makedirs(output_folder_hor, exist_ok=True)
    os.makedirs(output_folder_ver, exist_ok=True)

    images = [img for img in os.listdir(image_folder) if img.endswith(('.png'))]
    total_images = len(images)
    
    num_images_to_augment = int(total_images * percentage)

    selected_images_hor = random.sample(images, num_images_to_augment)

    remaining_images = [img for img in images if img not in selected_images_hor]
    selected_images_ver = random.sample(remaining_images, num_images_to_augment)

    for img_name in selected_images_hor:
        img_path = os.path.join(image_folder, img_name)
        img = Image.open(img_path)
        img_flipped = img.transpose(Image.FLIP_LEFT_RIGHT)
        new_img_name = img_name.replace('.', 'fx.')
        img_flipped.save(os.path.join(output_folder_hor, new_img_name))

    for img_name in selected_images_ver:
        img_path = os.path.join(image_folder, img_name)
        img = Image.open(img_path)
        img_flipped = img.transpose(Image.FLIP_TOP_BOTTOM)
        new_img_name = img_name.replace('.', 'fy.')
        img_flipped.save(os.path.join(output_folder_ver, new_img_name))

input_folder = '../data/images/train'
output_folder_hor = '../data/images/fx'
output_folder_ver = '../data/images/fy'

flip_images(input_folder, output_folder_hor, output_folder_ver)


In [18]:
# count & confirm percentages

def count_files(folder_path):
    files = os.listdir(folder_path)
    file_count = len([file for file in files if os.path.isfile(os.path.join(folder_path, file))])
    return file_count

directory_org = '../data/images/train'
directoryx = '../data/images/fx'
directoryy = '../data/images/fy'


print("Number of files in the label:", count_files(directory_org))
print("Number of files in the image:", count_files(directoryx))
print("Number of files in the image:", count_files(directoryy))



Number of files in the label: 23478
Number of files in the image: 3521
Number of files in the image: 3521


In [20]:
# modify  annotations for flipped images FX

def adjust_labels_for_flipping(image_directory, label_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    flipped_images = [f for f in os.listdir(image_directory) if f.endswith('fx.png')]

    for image_name in flipped_images:
        image_id = image_name.replace('fx.png', '')

        # construct the original and new label paths
        original_label_path = os.path.join(label_directory, f'{image_id}.txt')
        new_label_path = os.path.join(output_directory, f'{image_id}fx.txt')

        if os.path.exists(original_label_path):
            with open(original_label_path, 'r') as file:
                lines = file.readlines()
            
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                class_id = parts[0]
                x_center = str(1 - float(parts[1]))  # flip x_center horizontally
                y_center = parts[2]  # y_center 
                width = parts[3]  # width remains 
                height = parts[4]  # height remains 

                # create the new label line and add to new_lines
                new_line = f"{class_id} {x_center} {y_center} {width} {height}\n"
                new_lines.append(new_line)
            
            # write the new lines to the new label file
            with open(new_label_path, 'w') as file:
                file.writelines(new_lines)
        else:
            print(f"Label file does not exist for {image_id}, skipping.")


image_directory = '../data/images/fy'
label_directory = '../data/labels/train'
output_directory = '../data/labels/fyl'

adjust_labels_for_flipping(image_directory, label_directory, output_directory)


In [24]:
# Modify  annotations for flipped images FY

def adjust_labels_for_flipping(image_directory, label_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    flipped_images = [f for f in os.listdir(image_directory) if f.endswith('fy.png')]

    for image_name in flipped_images:
        image_id = image_name.replace('fy.png', '')

        # construct the original and new label paths
        original_label_path = os.path.join(label_directory, f'{image_id}.txt')
        new_label_path = os.path.join(output_directory, f'{image_id}fy.txt')

        if os.path.exists(original_label_path):
            with open(original_label_path, 'r') as file:
                lines = file.readlines()
            
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                class_id = parts[0]
                x_center = parts[1]  # x_center remains 
                y_center = str(1 - float(parts[2]))  # flip y_center vertically
                width = parts[3]  # width remains 
                height = parts[4]  # height remains 

                # create the new label line and add to new_lines
                new_line = f"{class_id} {x_center} {y_center} {width} {height}\n"
                new_lines.append(new_line)
            
            # write the new lines to the new label file
            with open(new_label_path, 'w') as file:
                file.writelines(new_lines)
        else:
            print(f"Label file does not exist for {image_id}, skipping.")


image_directory = '../data/images/fy'
label_directory = '../data/labels/train'
output_directory = '../data/labels/fyl'

adjust_labels_for_flipping(image_directory, label_directory, output_directory)


In [25]:
import os

def count_files(folder_path):
    files = os.listdir(folder_path)
    file_count = len([file for file in files if os.path.isfile(os.path.join(folder_path, file))])
    return file_count

directory_org = '../data/labels/train'
directoryx = '../data/labels/fxl'
directoryy = '../data/labels/fyl'


print("Number of files in the label:", count_files(directory_org))
print("Number of files in the image:", count_files(directoryx))
print("Number of files in the image:", count_files(directoryy))



Number of files in the label: 23478
Number of files in the image: 3521
Number of files in the image: 3521


In [ ]:
# check the images w/ drawn bounding boxes

def draw_boxes_on_images(label_dir, image_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    
    for label_file in label_files:
        label_file_path = os.path.join(label_dir, label_file)
        
        # read the image file
        base_filename = os.path.splitext(label_file)[0]
        image_file_path = os.path.join(image_dir, f"{base_filename}.png")
        if not os.path.exists(image_file_path):
            continue  # skip 
        
        # Load the image
        image = cv2.imread(image_file_path)
        im_height, im_width = image.shape[:2]
        
        # read bounding boxes from label file
        with open(label_file_path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                parts = line.strip().split()
                class_id, x_center, y_center, width, height = map(float, parts)
                x_center, width = x_center * im_width, width * im_width
                y_center, height = y_center * im_height, height * im_height
                
                # convert from center coordinates to top left coordinates
                x1 = int(x_center - width / 2)
                y1 = int(y_center - height / 2)
                x2 = int(x_center + width / 2)
                y2 = int(y_center + height / 2)
                
                #draw rectangle on the image
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # save 
        output_image_path = os.path.join(output_dir, f"{base_filename}.png")
        cv2.imwrite(output_image_path, image)
    
    print({output_dir})


label_directory = '../data/labels/fyl'
image_directory = '../data/images/fy'
output_directory = '../data/images/fy_bound'

draw_boxes_on_images(label_directory, image_directory, output_directory)


In [ ]:
# move the augmented images to to train folder:

def move_augmented_images(source_directory, target_directory):
    images = [f for f in os.listdir(source_directory) if f.endswith(('.png'))]
    for img_name in images:
        source_path = os.path.join(source_directory, img_name)
        target_path = os.path.join(target_directory, img_name)

        shutil.move(source_path, target_path)
        print(f"Moved {img_name} to {target_directory}")

# source_directory = '../data/images/fx'
source_directory = '../data/images/fy'
target_directory = '../data/images/train'

move_augmented_images(source_directory, target_directory)


In [ ]:
def move_modified_labels(source_directory, target_directory):
    labels = [f for f in os.listdir(source_directory) if f.endswith('.txt')]

    for label in labels:
        source_path = os.path.join(source_directory, label)
        target_path = os.path.join(target_directory, label)

        shutil.move(source_path, target_path)
        print(f"Moved {label} to {target_directory}")

source_directory = '../data/labels/fxl'
source_directory = '../data/labels/fyl'
target_directory = '../data/labels/train'

move_modified_labels(source_directory, target_directory)


In [35]:
directory_im = '../data/images/train'
directory_im_val = '../data/images/val'
directory_la = '../data/labels/train'
directory_la_val = '../data/labels/val'


print("Number of files in the image:", count_files(directory_im))
print("Number of files in the image:", count_files(directory_im_val))
print("Number of files in the label:", count_files(directory_la))
print("Number of files in the label:", count_files(directory_la_val))

Number of files in the image: 30520
Number of files in the image: 2323
Number of files in the label: 30520
Number of files in the label: 2323
